In [ ]:
#!/usr/bin/env python3
"""
局部轨迹规划模块 (Layer 2) - 改进版
====================================
将几何路径转换为时间参数化轨迹

算法改进:
    1. 三次样条插值 (替代线性插值) - 更平滑的路径
    2. 基于优化的速度规划 - 考虑加速度约束
    3. 梯形速度曲线 - 平滑的加减速

功能:
    - 路径平滑 (三次样条)
    - 曲率计算
    - 基于曲率的速度规划 (弯道减速)
    - 梯形加减速处理
    - 时间参数化轨迹生成

输入: 几何路径 [(x, y), ...], 当前状态 [x, y, theta]
输出: 时间参数化轨迹 [{x, y, theta, v, omega, t}, ...]

作者: GitHub Copilot
"""

import numpy as np  # 引入 NumPy 用于数值计算
from scipy import interpolate   # 引入 SciPy 的插值模块，用于三次样条插值

# ROS 依赖 (可选)
try:
    import rospy
    HAS_ROS = True
except ImportError:
    HAS_ROS = False


def _log_info(msg):
    if HAS_ROS and not rospy.is_shutdown():
        rospy.loginfo(msg)
    else:
        print(f"[INFO] {msg}")


    generate()  【主入口】
      ├─→ _smooth_path_spline()  【B样条平滑】
      │    ├─→ _interpolate_path()  （备用线性插值）
      │    └─→ 【更新 _spline_headings, _spline_curvatures】
      ├─→ _compute_curvatures()  【计算曲率】
      ├─→ _plan_speeds_optimized()  【速度规划】
      │    └─→ _normalize_angle()  （角度处理）
      ├─→ _build_trajectory_improved()  【轨迹构建】
      │    └─→ _normalize_angle()  （角度处理）
      └─→ _add_stop_segment()  【添加停止段】

    get_reference_window()
      └─→ get_reference_at_time()  【时间查询】

# 公开方法